In [2]:
import numpy    as np 
from   gurobipy import GRB, Model

class Bug:
    bug_id = 0
    def __init__(self, bug_type:int, due_date:int):
        self.ID                 = Bug.bug_id
        Bug.bug_id             += 1
        self.bug_type           = bug_type # LDA Category
        self.due_date           = due_date
        self.days_remained_open = 0
        
    def postponed(self):
        self.days_remained_open += 1
        
    def __repr__(self):
        return str(self.ID)
    
class Developer:
    dev_id = 0
    def __init__(self, simultaneous_job:int, filled_capacity:list, experience:list, Total_limit:int):
        self.ID                  = Developer.dev_id
        Developer.dev_id        += 1
        self.simultaneous_job    = simultaneous_job
        self.filled_capacity     = filled_capacity
        self.Total_limit         = Total_limit # Time horizon
        self.assigned_bugs       = []
        self.time_limit          = [Total_limit-used for used in filled_capacity]
        self.experience          = experience # LDA_experience
        self.schedule            = self.update_schedule()    #T_{jt}^d
        
    def assign_a_bug(self, bug: Bug, which_slot: int):
        self.filled_capacity[which_slot] += Bug.fixing_time
        self.time_limit[which_slot]      -= Bug.fixing_time
        assert (self.filled_capacity[which_slot] <= Total_limit)
        self.assigned_bugs.append(bug.ID)
        
    def __repr__(self):
        return str(self.ID)
    
    def update_schedule(self):
        schedule = []
        for filled_capacity in (self.filled_capacity):
            schedule.append([0 if i<=filled_capacity else 1 for i in range(self.Total_limit)])
        return schedule
    
    
class State:
    def __init__(self, dev_dict:dict, bugs_dict:dict, time):
        self.devs_dict        = dev_dict
        self.bugs_dict        = bugs_dict
        self.time             = time
        self.S_t_d_dev        = {}
        self.S_t_b_bug        = {}
        self.state_update()
        
    def state_update(self):
        self.S_t_d_dev    = {}
        for dev in self.devs_dict.values():
            if tuple(dev.experience) not in self.S_t_d_dev:
                self.S_t_d_dev[tuple(dev.experience)] = 0
            self.S_t_d_dev[tuple(dev.experience)] += 1
        self.S_t_b_bug    = {}
        for bug in self.bugs_dict.values():
            if (bug.bug_type) not in self.S_t_b_bug:
                self.S_t_b_bug[(bug.bug_type)] = 0
            self.S_t_b_bug[(bug.bug_type)] += 1

class Decision:
    def __init__(self, system_state):
        self.pbt            = {} # num of bugs with attr b postponed to time t+1
        self.ydbt           = {} # num of bugs with attr b solved by dev with attr d
        self.model          = Model()
        self.feasible_actions(system_state)
        
    def feasible_actions(self, system_state, LogToConsole= False, TimeLimit=60, write=False):
        n_bugs                         = len(system_state.S_t_b_bug)
        n_devs                         = len(system_state.S_t_d_dev)        
        self.model.params.LogToConsole = LogToConsole
        self.model.params.TimeLimit    = TimeLimit
        for bug_type in range(n_bugs):
            self.pbt[bug_type] = (self.model.addVar(vtype= GRB.INTEGER, name=f'p[{bug_type}]'))
        for bug_type in range(n_bugs):
            for dev_type in range(n_devs):
                self.ydbt[bug_type,dev_type] = (self.model.addVar(vtype= GRB.INTEGER, name=f'y[{bug_type},{dev_type}]'))
                        
        self.model.setObjective(0, GRB.MINIMIZE)
        
        for dev_type, dev_d in enumerate(system_state.S_t_d_dev):
            n_assigned_devs_constraint = 0
            for bug_type in range(n_bugs):
                n_assigned_devs_constraint += self.ydbt[bug_type,dev_type]
            self.model.addConstr(n_assigned_devs_constraint <= system_state.S_t_d_dev[dev_d],
                                 name = "eq:assigned_devs")

        for bug_type, bug_b in enumerate(system_state.S_t_b_bug):
            n_assigned_bugs_constraint = 0
            for dev_type in range(n_devs):
                n_assigned_bugs_constraint += self.ydbt[bug_type,dev_type]
            n_assigned_bugs_constraint += self.pbt[bug_type]
            self.model.addConstr(n_assigned_bugs_constraint == system_state.S_t_b_bug[bug_b],
                                 name = "eq:assigned_bugs")
        if write:
            self.model.write('ADP.lp')
        self.model.optimize()
        

In [8]:
n_devs      = 5
Total_limit = 10
n_bug_types = 8
n_bugs      = 20


dev_dict    = {}
Developer.dev_id = 0
for dev_id in range(n_devs):
    np.random.seed(dev_id)
    simultaneous_job = np.random.choice([1,2,3], p=[0.8, 0.15, 0.05])
    filled_capacity  = [np.random.choice(range(1,7)) for i in range(simultaneous_job)]
    experience       = [np.random.choice([1,1,2,2,3,4,4,5,6]) for i in range(n_bug_types)]
    dev_dict[dev_id] = Developer(simultaneous_job, filled_capacity, experience, Total_limit)

bug_dict    = {}
Bug.bug_id = 0
for bug_id in range(n_bugs):
    np.random.seed(bug_id)
    bug_dict[bug_id] = Bug(bug_type=np.random.choice(range(n_bug_types)),
                           due_date=np.random.choice([10,8,11,5,7,8,10,11,3,20]))
    bug_dict[bug_id].days_remained_open = np.random.choice([5,10,7,8,12,25,7,5,6,5,8,2,0,0,0,0,1])

In [4]:
system_state = State(dev_dict, bug_dict, time=0)

In [5]:
decision_vars = Decision(system_state)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-02-02


In [6]:
decision_vars.__dict__

{'pbt': {0: <gurobi.Var p[0] (value 1.0)>,
  1: <gurobi.Var p[1] (value 2.0)>,
  2: <gurobi.Var p[2] (value 2.0)>,
  3: <gurobi.Var p[3] (value 5.0)>,
  4: <gurobi.Var p[4] (value 4.0)>,
  5: <gurobi.Var p[5] (value 2.0)>,
  6: <gurobi.Var p[6] (value 1.0)>,
  7: <gurobi.Var p[7] (value 3.0)>},
 'ydbt': {(0, 0): <gurobi.Var y[0,0] (value 0.0)>,
  (0, 1): <gurobi.Var y[0,1] (value 0.0)>,
  (0, 2): <gurobi.Var y[0,2] (value 0.0)>,
  (0, 3): <gurobi.Var y[0,3] (value 0.0)>,
  (0, 4): <gurobi.Var y[0,4] (value 0.0)>,
  (1, 0): <gurobi.Var y[1,0] (value 0.0)>,
  (1, 1): <gurobi.Var y[1,1] (value 0.0)>,
  (1, 2): <gurobi.Var y[1,2] (value 0.0)>,
  (1, 3): <gurobi.Var y[1,3] (value 0.0)>,
  (1, 4): <gurobi.Var y[1,4] (value 0.0)>,
  (2, 0): <gurobi.Var y[2,0] (value 0.0)>,
  (2, 1): <gurobi.Var y[2,1] (value 0.0)>,
  (2, 2): <gurobi.Var y[2,2] (value 0.0)>,
  (2, 3): <gurobi.Var y[2,3] (value 0.0)>,
  (2, 4): <gurobi.Var y[2,4] (value 0.0)>,
  (3, 0): <gurobi.Var y[3,0] (value 0.0)>,
  (3, 1)

In [14]:
bug_dict

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 8: 8,
 9: 9,
 10: 10,
 11: 11,
 12: 12,
 13: 13,
 14: 14,
 15: 15,
 16: 16,
 17: 17,
 18: 18,
 19: 19}